In [ ]:
import pandas
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

#IMPORTANDO DADOS DAS POLARES OBTIDAS NO XFLR5 E DATCOM
#Polares da asa
df = pandas.read_excel('WING_polars.xlsx', skiprows=7)
#print("Dados disponíveis da asa: \n",df.columns) #print the column names
alpha_w = df['alpha'].values #armazena os valores da coluna em um array de 1 coluna
CL_w = df['CL'].values
CD_w = df['CD'].values
Cm_w = df['Cm'].values
XCP_w = df['XCP'].values

#polares
plt.figure()
plt.plot(alpha_w,CL_w)
#plt.show()

plt.figure()
plt.plot(alpha_w,CD_w)
#plt.show()

plt.figure()
plt.plot(alpha_w,Cm_w)
#plt.show()

#Polares do HT
df1 = pandas.read_excel('HT_polars.xlsx', skiprows=7)
#print("Dados disponíveis do HT: \n",df1.columns) #print the column names
alpha_t = df1['alpha'].values
CL_t = df1['CL'].values
CD_t = df1['CD'].values
Cm_t = df1['Cm'].values
XCP_t = df1['XCP'].values

#polares
plt.figure()
plt.plot(alpha_t,CL_t)
#plt.show()

plt.figure()
plt.plot(alpha_t,CD_t)
#plt.show()

plt.figure()
plt.plot(alpha_t,Cm_t)
#plt.show()


#Polares do conjunto ASA HT
df2 = pandas.read_excel('ALL_polars.xlsx', skiprows=7)
#print("Dados disponíveis do conjunto: \n",df2.columns) #print the column names
alpha_c = df2['alpha'].values
CL_c = df2['CL'].values
CD_c = df2['CD'].values
Cm_c = df2['Cm'].values
XCP_c = df2['XCP'].values

#Polares da fuselagem (DATCOM)
df3 = pandas.read_excel('Datcom.xlsx', skiprows=0)
#print("Dados disponíveis da fusela: \n",df2.columns) #print the column names
alpha_f = df3['ALPHA'].values
CL_f = df3['CL'].values
CD_f = df3['CD'].values
Cm_f = df3['CM'].values
XCP_f = df3['XCP'].values

#polares
plt.figure()
plt.plot(alpha_f,CL_f)
#plt.show()

plt.figure()
plt.plot(alpha_f,CD_f)
#plt.show()

plt.figure()
plt.plot(alpha_f,Cm_f)
#plt.show()


# Características da asa (sistema ìmperial)
wing_b = 33.8                  #Envergadura
wing_S = 182                   #Área
wing_AR = wing_b**2 / wing_S   #alongamento
wing_cr = 6.12                 #corda na raíz
wing_ct = 4.41                 #corda na ponta
wing_Lambda = 0.35             #enflechamento
wing_lambda = wing_ct/wing_cr  #afilamento
wing_iw = 0                    #ang de incidência
wing_a_l_0 = -2                #ângulo com lift 0
wing_e = 1.78*(1-0.045*wing_AR**0.68)-0.64             #fator de Oswald
wing_k = 1 / (np.pi * wing_AR * wing_e)                #parâmetro da polar de arrasto
wing_cd0 = 0.006                #arrasto parasita


# Características do HT
ht_cr = 3.13
ht_ct = 2.59
ht_Lambda = 7.56
ht_b = 13.18
ht_S = (ht_cr + ht_ct) * ht_b * 0.5
ht_it = 0
ht_a_l_0 = 0
ht_a0 = 0.063
ht_AR = ht_b**2 / ht_S
ht_e = 1.78*(1-0.045*ht_AR**0.68)-0.64
ht_k = 1 / (np.pi * ht_AR * ht_e)
ht_cd0 = 0.005

#pegando o Cm pelo datcom
df3 = pandas.read_excel('Datcom.xlsx', skiprows=0)
Cm_f = df3['CM'].values
Cl_f = df3['CL'].values

deflexoes = glob('Cessna*.txt') #seleciona os txts com as polares para cada deflexão do conjunto asa+HT
#Cria uma lista com os elementos da lista sendo as matrizes contendo as polares para cada deflexão
DadosDef = [pandas.read_csv(f, delimiter="\s+", skiprows=7) for f in deflexoes]

deflexoesHT = glob('Cessna*.txt') #seleciona os txts com as polares para cada deflexão
#Cria uma lista com os elementos da lista sendo as matrizes contendo as polares para cada deflexão
DadosDefHT = [pandas.read_csv(f, delimiter="\s+", skiprows=7) for f in deflexoesHT]

plt.figure()


###TRABALHANDO OS DADOS DAS TABELAS

#-------Deflexão de trimagem x Cl trimado--------

ClalphaW = (CL_w[30]-CL_w[29])/0.5
ClalphaT = (CL_t[30]-CL_t[29])/0.5
ClalphaC = (CL_c[30]-CL_c[29])/0.5

Trim = [-20, -15, -10, -5, 5, 10, 15]
Cltrim = [1.186,0.934, 0.674, 0.365, -0.261,-0.572, -0.875]

plt.plot(Cltrim, Trim, 'b')
plt.ylabel(r'Deflexão de trimagem (graus)')
plt.xlabel(r'CL trimado')
plt.grid('on')
plt.show()


#--------Downwash por alpha------
Cl0_c = 0.140128
Cl0_t = 0.000964
Cl0_w = 0.153994

Clalphas = ClalphaW+ ClalphaT-ClalphaC
Cl0s = Cl0_w+Cl0_t-Cl0_c

Ang_downwash = ((Clalphas)*alpha_w+(Cl0s))/(ClalphaT)

plt.plot(alpha_w,Ang_downwash)
plt.ylabel(r'epslon')
plt.xlabel(r'$\alpha$')
plt.grid('on')
#plt.show()

#------- Centros aerodinâmicos obtidos pelo XFLR5
Ca_w = 18
Ca_ht = 194.8
Ca_f = -6

#--------Ponto netro-------------
ClalphaF = (CL_f[30]-CL_f[29])/0.5
Xw = 1
Xf = 1
Xht = 1
EpslonAlpha = (Ang_downwash[10]-Ang_downwash[9])/5
X_Pneutro = (ClalphaF*Ca_w + ClalphaT*ht_S/wing_S*(1-EpslonAlpha)*Ca_ht+ClalphaF*Ca_f)/(ClalphaF+ClalphaT*ht_S/wing_S*(1-EpslonAlpha)+ClalphaF)


#TRABALHANDO COM HT
plotando = plt.subplot(131)
plotando1 = plt.subplot(132)
plotando2 = plt.subplot(133)
i=-30
for a in DadosDefHT:
    alphad = a['alpha'].values  # armazena os valores da coluna em um array de 1 coluna
    CLd = a['CL'].values
    Cmd = a['Cm'].values
    Cdd = a['CD'].values

    #valor = len(alphad)
    #soma = Cmd + Cm_f[0:valor]
    label = (r"deflexão: ", i)
    plotando.plot(alphad, CLd ,  label = label)
    plotando1.plot(alphad, Cmd, label =   label)
    plotando2.plot(alphad, Cdd, label=label)
    i += 5

plt.title('CmCg por alpha')
plt.ylabel(r'CmCg')
plt.xlabel(r'alpha')

plt.grid('on')
plt.legend(loc='best')
#plt.show()

plt.figure()



i = -30
for a in DadosDef:
    alphad = a['alpha'].values  # armazena os valores da coluna em um array de 1 coluna
    CLd = a['CL'].values
    Cmd = a['Cm'].values
    XCPd = a['XCP'].values

    valor = len(Cmd)
    soma = Cmd + Cm_f[0:valor]
    label = (r"deflexão: ", i)
    plt.plot(alphad, soma ,  label = label)
    i += 5

plt.title('CmCg por alpha para diferentes deflexões')
plt.ylabel(r'CmCg')
plt.xlabel(r'alpha')

plt.grid('on')
plt.legend(loc='best')
#plt.show()

plt.figure()

i = -30

for a in DadosDef:
    alphad = a['alpha'].values  # armazena os valores da coluna em um array de 1 coluna
    CLd = a['CL'].values
    Cmd = a['Cm'].values
    XCPd = a['XCP'].values
    valor = len(Cmd)

    soma = Cmd + Cm_f[0:valor]
    soma1 = CLd + Cl_f[0:valor]

    cbarra = 66.431         #corda media aerodinâmica
    Xcg = 17.7
    Xcp = Xcg - cbarra*soma/soma1
    Xcpd = interp1d(alphad, XCPd, kind='cubic')
    label = (r"deflexão: ", i)
    plt.plot( np.linspace(-8,8,50), Xcpd(np.linspace(-8,8,50)),'.')
    i += 5

plt.title('Xcp x alpha')
plt.ylabel(r'Xcp (in)')
plt.xlabel(r'alpha')
plt.grid('on')
plt.legend(loc='best')
#plt.show()

soma =[]
soma2 =[]
cas = 0
diferenca = []
diferenca1 = []
plin = 0

for a in DadosDef:
    alphad = a['alpha'].values  # armazena os valores da coluna em um array de 1 coluna
    CLd = a['CL'].values
    Cmd = a['Cm'].values  #+25 +30
    XCPd = a['XCP'].values

    if len(alphad)==57 and cas>4:
        somas = CLd+Cl_f
        somas1 = Cmd+Cm_f
        soma.append(somas)
        soma2.append(somas1)
        if plin>1:
         diferenca.append(abs(soma[-1][5] - soma[plin-1][5])/5)
         diferenca1.append(abs(soma2[-1][5] - soma2[plin-1][5])/5)
        plin+=1
    cas += 1

print(' Derivada de controle de CL: ', np.mean(diferenca))
print(' Derivade de controle de Cm: ', np.mean(diferenca1))



#------- Cálculo da margem estática
Xcg = Xcg+np.linspace(-10,50)
Xcg = Xcg/12*0.3048*100 #conversão polegadas para centimetros

Ms = (X_Pneutro-Xcg)/cbarra

plt.figure()
#label = (r"deflexão: ", i)
plt.plot(Xcg, Ms, 'b')
plt.title('Margem estática por posição do cg')
plt.ylabel(r'Margem estática')
plt.xlabel(r'Xcg (cm)')
plt.grid('on')
plt.legend(loc='best')
plt.show()

#colourmap
